<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/final_project_comp_ling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project**
---


*VU Introduction to Computational Linguistics - WS24/25*

---
**Topic**: Toxicity Detection in Social Media  
Detecting toxic or offensive language in social media, e.g. YouTube comments in Polish and English    

**Datasets**: PolishCyberbullyingDataset or hatecheck-polish for the Polish dataset and civil_comments or toxicity_classification_jigsaw for the English dataset
model: XLM-RoBERTa Base   



**Set-Up:**
Installing librarires required for the task

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install matplotlib seaborn
!pip install optuna
!pip install tensorboard
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.8 MB/s eta 0:00:00

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, DatasetDict

# English dataset (civil_comments)
dataset_en = load_dataset("civil_comments")

# Polish dataset (PolishCyberbullyingDataset)
dataset_pl = load_dataset("ptaszynski/PolishCyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

v2_training_anonymized.tsv:   0%|          | 0.00/801k [00:00<?, ?B/s]

v2_test_anonymized.tsv:   0%|          | 0.00/79.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10038 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset_en)
print(dataset_pl)

DatasetDict({
    train: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 1804874
    })
    validation: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
    test: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
})
DatasetDict({
    train: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 10038
    })
    test: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 1000
    })
})


In [ ]:
train_valid_split = dataset_pl['train'].train_test_split(test_size=0.1, seed=42)

In [ ]:
train_en = dataset_en['train']
val_en = dataset_en['validation']
test_en = dataset_en['test']

train_pl = train_valid_split['train']
val_pl = train_valid_split['test']
test_pl = dataset_pl['test']

In [ ]:
print("Train size:", len(train_pl))
print("Validation size:", len(val_pl))
print("Test size:", len(test_pl))

print("Train size:", len(train_en))
print("Validation size:", len(val_en))
print("Test size:", len(test_en))

Train size: 9034
Validation size: 1004
Test size: 1000
Train size: 1804874
Validation size: 97320
Test size: 97320


Preprocessing!

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
  input_column = 'text' if 'text' in examples else 'TEXT'
  return tokenizer(examples[input_column], truncation=True, padding="max_length", max_length=128, return_tensors="pt")

# tokenizing dataset
tokenized_train_en = train_en.map(tokenize_function, batched=True)
tokenized_val_en = val_en.map(tokenize_function, batched=True)
tokenized_test_en = test_en.map(tokenize_function, batched=True)

tokenized_train_pl = train_pl.map(tokenize_function, batched=True)
tokenized_val_pl = val_pl.map(tokenize_function, batched=True)
tokenized_test_pl = test_pl.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # for dynamic padding during training

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/9034 [00:00<?, ? examples/s]

Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# English dataset: map 'toxicity' to 'labels' (0: non-toxic, > 0.5: toxic)
tokenized_train_en = tokenized_train_en.map(lambda x: {"labels": int(x["toxicity"] > 0.5)})
tokenized_val_en = tokenized_val_en.map(lambda x: {"labels": int(x["toxicity"] > 0.5)})
tokenized_test_en = tokenized_test_en.map(lambda x: {"labels": int(x["toxicity"] > 0.5)})

# Polish dataset: map 'ANY PROFANITIES' to 'labels' (0: non-toxic, 1: toxic)
tokenized_train_pl = tokenized_train_pl.map(lambda x: {"labels": 1 if x["GENERAL TAG"] == 1 else 0})
tokenized_val_pl = tokenized_val_pl.map(lambda x: {"labels": 1 if x["GENERAL TAG"] == 1 else 0})
tokenized_test_pl = tokenized_test_pl.map(lambda x: {"labels": 1 if x["GENERAL TAG"] == 1 else 0})

Map:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/9034 [00:00<?, ? examples/s]

Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Define save paths in your Drive
save_path = '/content/drive/MyDrive/tokenized_datasets/'

# Save English datasets
tokenized_train_en.save_to_disk(save_path + 'tokenized_train_en')
tokenized_val_en.save_to_disk(save_path + 'tokenized_val_en')
tokenized_test_en.save_to_disk(save_path + 'tokenized_test_en')

# Save Polish datasets
tokenized_train_pl.save_to_disk(save_path + 'tokenized_train_pl')
tokenized_val_pl.save_to_disk(save_path + 'tokenized_val_pl')
tokenized_test_pl.save_to_disk(save_path + 'tokenized_test_pl')

Saving the dataset (0/4 shards):   0%|          | 0/1804874 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9034 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1004 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
# loading my datasets
from datasets import load_from_disk

# Load English datasets
tokenized_train_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_train_en')
tokenized_val_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_val_en')
tokenized_test_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_test_en')

# Load Polish datasets
tokenized_train_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_train_pl')
tokenized_val_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_val_pl')
tokenized_test_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_test_pl')

In [4]:
# Reduce training dataset to 10,000 samples
small_train_en = tokenized_train_en.shuffle(seed=42).select(range(10000))

# Reduce validation dataset to 2,000 samples
small_val_en = tokenized_val_en.shuffle(seed=42).select(range(2000))

# Reduce test dataset to 2,000 samples
small_test_en = tokenized_test_en.shuffle(seed=42).select(range(2000))

finetuning XLM-RoBERT Base

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load XLM-RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=2  # Binary classification
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainingArguments --> for better control of training behaviour

In [6]:
# for handling of variable-length sequences in batchesv
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [8]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import evaluate
import numpy as np

In [9]:
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="./results", # where to put everything that is trained
    per_device_train_batch_size=8,  # amount of sentences model looks at before anything is updated
    per_device_eval_batch_size=8,
    logging_steps=10,                # how many times results are logged/track/output
    num_train_epochs=10,
    eval_strategy="epoch",          # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,             # influences how weight changes in function, determines how big a step you take in valley
    weight_decay=0.01,              # sum of weights added to loss function, regulization method, penalizes large weights
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,             # Keep only 2 checkpoints to save space# Metric to select the best model
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer_en = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_train_en,  # Training data
    eval_dataset=small_val_en,     # Validation data
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_en.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

<ipython-input-9-692bcbe46c74>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_en = Trainer(


In [ ]:
trainer_en.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.213600,0.192758,0.952500
2,0.192400,0.218453,0.942000
3,0.218900,0.215010,0.950500


TrainOutput(global_step=3750, training_loss=0.19061492944260439, metrics={'train_runtime': 1153.989, 'train_samples_per_second': 86.656, 'train_steps_per_second': 10.832, 'total_flos': 1976490247864320.0, 'train_loss': 0.19061492944260439, 'epoch': 3.0})

In [10]:
trainer_pl = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_train_pl,  # Training data
    eval_dataset=tokenized_val_pl,     # Validation data
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_pl.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

<ipython-input-10-4ff39c44d4c8>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_pl = Trainer(


In [ ]:
trainer_pl.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.469100,0.373096,0.860558
2,0.405700,0.367718,0.855578
3,0.516900,0.385694,0.863546
4,0.240100,0.406184,0.867530
5,0.523900,0.463268,0.868526
6,0.513900,0.462030,0.858566
7,0.581600,0.400516,0.870518
8,0.321700,0.478002,0.870518
9,0.174300,0.456222,0.869522


TrainOutput(global_step=10170, training_loss=0.32110017780709055, metrics={'train_runtime': 3098.1167, 'train_samples_per_second': 29.16, 'train_steps_per_second': 3.647, 'total_flos': 5348126866775040.0, 'train_loss': 0.32110017780709055, 'epoch': 9.0})

**Defining Objective Function**
--> defines training process and the hyperparameter to optimize

In [14]:
import optuna

# Define the objective function
def objective(trial):
    # Suggest hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)  # Log scale
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])  # Choose from options
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",                     # Output directory
        eval_strategy="epoch",               # Evaluate after every epoch
        save_strategy="epoch",                     # Save model after each epoch
        learning_rate=learning_rate,               # Tuned learning rate
        per_device_train_batch_size=batch_size,    # Tuned batch size
        per_device_eval_batch_size=batch_size,     # Same as train batch size
        num_train_epochs=5,                        # Number of epochs
        weight_decay=weight_decay,                 # Tuned weight decay
        logging_dir="./logs",                      # Logging directory
        logging_steps=10,                          # Log every 10 steps
        load_best_model_at_end=True,               # Load best model
        save_total_limit=1,                        # Save only the best checkpoint
        seed=42,                                   # Reproducibility
        report_to="none"
    )

    # Define the Trainer
    trainer = Trainer(
        model=model,                       # Pre-trained model
        args=training_args,                # Training arguments
        train_dataset=tokenized_train,     # Replace with tokenized_train_en or tokenized_train_pl
        eval_dataset=tokenized_val,        # Replace with tokenized_val_en or tokenized_val_pl
        processing_class=tokenizer,               # Tokenizer
        data_collator=data_collator,       # Data collator
        compute_metrics=compute_metrics    # Metrics function
    )

    # Add EarlyStoppingCallback if desired
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

    # Train the model and return the best validation metric
    result = trainer.train()
    metrics = trainer.evaluate()  # Evaluate on validation set
    return metrics["eval_accuracy"]  # Return validation accuracy as the objective

Running Optuna for the English Dataset

In [ ]:
# Tokenized English datasets
tokenized_train = small_train_en
tokenized_val = small_val_en

# Run Optuna optimization
study_en = optuna.create_study(direction="maximize")  # Maximize validation accuracy
study_en.optimize(objective, n_trials=10)  # Number of trials to run

# Print the best hyperparameters for English
print("Best hyperparameters for English:")
print(study_en.best_params)

[I 2025-01-27 10:33:55,506] A new study created in memory with name: no-name-07d4e7a8-a2fb-4240-853d-6bd507d1ba2b


Epoch,Training Loss,Validation Loss,Accuracy
1,0.203200,0.146212,0.943500
2,0.109900,0.158363,0.953000
3,0.119700,0.187457,0.946000


[I 2025-01-27 10:49:16,758] Trial 0 finished with value: 0.9435 and parameters: {'learning_rate': 1.4872583278516296e-05, 'batch_size': 32, 'weight_decay': 0.09765206860701765}. Best is trial 0 with value: 0.9435.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.208100,0.187395,0.930500
2,0.119400,0.166462,0.956500
3,0.130000,0.208697,0.942500
4,0.041100,0.233401,0.947000


[I 2025-01-27 11:13:03,114] Trial 1 finished with value: 0.9565 and parameters: {'learning_rate': 2.248855674433608e-05, 'batch_size': 32, 'weight_decay': 0.09427773638474078}. Best is trial 1 with value: 0.9565.


...

[I 2025-01-27 12:07:24,086] Trial 3 finished with value: 0.9425 and parameters: {'learning_rate': 4.574636336227165e-05, 'batch_size': 16, 'weight_decay': 0.06167101575720779}. Best is trial 1 with value: 0.9565.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.461100,0.221305,0.943500


Epoch,Training Loss,Validation Loss,Accuracy
1,0.461100,0.221305,0.943500
2,0.174800,0.254712,0.943500
3,0.342100,0.228934,0.943500


[I 2025-01-27 12:30:24,734] Trial 4 finished with value: 0.9435 and parameters: {'learning_rate': 4.334055604111631e-05, 'batch_size': 8, 'weight_decay': 0.04824818350576944}. Best is trial 1 with value: 0.9565.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.462500,0.219959,0.943500
2,0.174000,0.254807,0.943500
3,0.341800,0.229005,0.943500


Running Optuna for the Polish Dataset

In [ ]:
# Tokenized Polish datasets
tokenized_train = tokenized_train_pl
tokenized_val = tokenized_val_pl

# Run Optuna optimization
study_pl = optuna.create_study(direction="maximize")  # Maximize validation accuracy
study_pl.optimize(objective, n_trials=10)  # Number of trials to run

# Print the best hyperparameters for Polish
print("Best hyperparameters for Polish:")
print(study_pl.best_params)

In [ ]:
# Saving Optuna Study in Drive

import pickle

# Save English study to Google Drive
with open("/content/drive/MyDrive/optuna_english_study.pkl", "wb") as f:
    pickle.dump(study_en, f)

# Save Polish study to Google Drive
with open("/content/drive/MyDrive/optuna_polish_study.pkl", "wb") as f:
    pickle.dump(study_pl, f)

In [ ]:
# Loading Saved Studies

with open("/content/drive/MyDrive/optuna_english_study.pkl", "rb") as f:
    study_en = pickle.load(f)

with open("/content/drive/MyDrive/optuna_polish_study.pkl", "rb") as f:
    study_pl = pickle.load(f)